# Flight engine predictive maintenance
Video: https://vimeo.com/160024508

Data source: https://c3.nasa.gov/dashlink/resources/140/

In [1]:
IS_READ_DATA = False      #flag if data is already loaded from CSV files into SQLite

## Read flight data

/data

    /Fault_X                        -> faulty equipments (X = Fan, HPC, HPT, LPC, LPT)
    /Nominal_X                      -> equipments without fault
    
        /EngineYYY                  -> engine number (3 digits)
        
            /FlightZZZ.csv          -> sensor data (sampled every second)
            /EngineHealth.csv       -> engine health data (each row = 1 flight)
            /Engine_Fuel_Effic.csv  -> engine fuel efficiency (each row = 1 flight)

First, read the data folder structure as a list of tuples:
**(path_to_folder, equipment_name (str), is_faulty (int), engine_number (int), list_of_files)**

In [2]:
from os import walk
data_path = '/nasa'

f = [(dirpath.replace("\\","/"), \
      dirpath.split("/")[-2].split("_")[-1],                 #equipment \
      True if "Fault" in dirpath.split("/")[-2] else False,  #is fault? \
      int(dirpath.split("/")[-1].split("Engine", 1)[1]),     #engine num
      sorted(filenames)
      #filter(lambda fn: ".csv" in fn, filenames)            #list of csv files
     )
     for (dirpath, dirnames, filenames) in walk(data_path) \
     if ("Fault_" in dirpath or "Nominal_" in dirpath) \
     and ("Engine" in dirpath) \
     and ("#extra" not in dirpath) \
     and len(dirnames) is 0]

print f

[('/nasa/data/Fault_Fan/Engine06', 'Fan', True, 6, ['EngineHealth.csv', 'Engine_Fuel_Effic.csv', 'Flight001.csv', 'Flight002.csv', 'Flight003.csv', 'Flight004.csv', 'Flight005.csv', 'Flight006.csv', 'Flight007.csv', 'Flight008.csv', 'Flight009.csv', 'Flight010.csv', 'Flight011.csv', 'Flight012.csv', 'Flight013.csv', 'Flight014.csv', 'Flight015.csv', 'Flight016.csv', 'Flight017.csv', 'Flight018.csv', 'Flight019.csv', 'Flight020.csv', 'Flight021.csv', 'Flight022.csv', 'Flight023.csv', 'Flight024.csv', 'Flight025.csv', 'Flight026.csv', 'Flight027.csv', 'Flight028.csv', 'Flight029.csv', 'Flight030.csv', 'Flight031.csv', 'Flight032.csv', 'Flight033.csv', 'Flight034.csv', 'Flight035.csv', 'Flight036.csv', 'Flight037.csv', 'Flight038.csv', 'Flight039.csv', 'Flight040.csv', 'Flight041.csv', 'Flight042.csv', 'Flight043.csv', 'Flight044.csv', 'Flight045.csv', 'Flight046.csv', 'Flight047.csv', 'Flight048.csv', 'Flight049.csv', 'Flight050.csv', 'Flight051.csv', 'Flight052.csv', 'Flight053.csv', 'F

Load data into Spark RDDs

In [3]:
from pyspark.sql import HiveContext, Row, Window
from pyspark.sql.types import *
import pyspark.sql.functions as fsql
import pandas

sqlContext = SQLContext(sc) #alternatively, use SQLContext(sc); SQLContext does not allow window functions

def procCsv(path):
    ditem = pandas.read_csv(path)
    return ditem

In [134]:
equip_df.head()

time  alt        MN      TRA      Wf     Fn   SmHPC   SmLPC  \
fl_num fl_time                                                                 
1      0        0.000  0.0  0.000000   0.0000  1.3797  18116  36.568  7.5047   
       1        1.005  0.0  0.006579   2.6316  1.3801  17791  36.559  7.5053   
       2        2.010  0.0  0.013191   5.2763  1.3820  17479  36.546  7.5053   
       3        3.015  0.0  0.019803   7.9211  1.5087  17944  34.933  7.8705   
       4        4.020  0.0  0.026414  10.5660  1.6288  19378  34.220  8.1368   

                 SmFan     T48   ...        NfR     NcR     BPR      farB  \
fl_num fl_time                   ...                                        
1      0        5.1714  1283.4   ...     1259.7  7448.9  9.4616  0.013074   
       1        5.1811  1283.5   ...     1259.6  7448.9  9.4646  0.013078   
       2        5.1985  1284.1   ...     1259.7  7449.0  9.4706  0.013093   
       3        5.3680  1323.5   ...     1277.1  7472.9  9.3710  0.013953   
       4        5.6938  1353.1   ...     1320.5  7507.4  9.4188  0.014458   

                htBleed  PCNfRdmd     W31      W32   eq  isfault  
fl_num fl_time                                                    
1      0         282.28    48.000  16.489   9.8933  Fan     None  
       1         282.28    49.946  16.490   9.8939  Fan     None  
       2         282.29    51.903  16.493   9.8955  Fan     None  
       3         285.23    53.859  16.894  10.1360  Fan     None  
       4         289.42    55.810  17.602  10.5610  Fan     None  

[5 rows x 32 columns]

In [135]:
#Correct time column
equip_df['time'] = equip_df['time'].apply(lambda f: int(f/1.005))

In [136]:
#For validation, display moment of occurence of fault (if any)
equip_df.query('isfault == True')[1:2]

,,time,alt,MN,TRA,Wf,Fn,SmHPC,SmLPC,SmFan,T48,...,NfR,NcR,BPR,farB,htBleed,PCNfRdmd,W31,W32,eq,isfault
fl_num,fl_time,,,,,,,,,,,,,,,,,,,,,
39,1036,1036,20627.0,0.68997,94.986,3.4821,23412,24.444,9.443,16.516,1857.8,...,2348.7,8035.9,9.3653,0.023612,355.86,98.337,23.042,13.825,Fan,True


# Feature engineering

Create feature **iscruise** - True if the altitude is 35000 feet

In [137]:
from numpy import where
equip_df['iscruise'] = where(equip_df['alt']==35000, True, False)

Group by 5-second windows and compute secondary parameters

In [138]:
from numpy import number
numeric_df = equip_df.select_dtypes(include=[number])

In [291]:
for fl_num in set(map(lambda (fl_num, pos): fl_num, numeric_df.index.values)):
    one_df = numeric_df.ix[fl_num]
    one_df['twindow'] = one_df['time'] // 5
    #a = one_df.rolling(window=10).mean()
    a = one_df.groupby(['twindow']).corr()
    break

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [295]:
a.head()

BPR        Fn        MN        Nc       NcR        Nf  \
twindow                                                                   
0       BPR  1.000000 -0.339781 -0.668873 -0.578808 -0.614291 -0.510475   
        Fn  -0.339781  1.000000  0.580523  0.914101  0.901422  0.932776   
        MN  -0.668873  0.580523  1.000000  0.858580  0.869178  0.834536   
        Nc  -0.578808  0.914101  0.858580  1.000000  0.999011  0.996644   
        NcR -0.614291  0.901422  0.869178  0.999011  1.000000  0.992019   

                  NfR       P15        P2       P30    ...          TRA  \
twindow                                                ...                
0       BPR -0.510890 -0.518604 -0.630740 -0.621847    ...    -0.668860   
        Fn   0.932685  0.926989  0.777709  0.897848    ...     0.580555   
        MN   0.834693  0.843197  0.962041  0.872046    ...     1.000000   
        Nc   0.996683  0.997397  0.963595  0.998523    ...     0.858600   
        NcR  0.992080  0.993248  0.967773  0.999950    ...     0.869195   

                  W31       W32        Wf  alt       epr      farB   htBleed  \
twindow                                                                        
0       BPR -0.576128 -0.575874 -0.692383  NaN -0.652224 -0.765678 -0.616654   
        Fn   0.914305  0.914467  0.863106  NaN  0.897843  0.814246  0.900593   
        MN   0.858658  0.858464  0.889479  NaN  0.860256  0.898994  0.869704   
        Nc   0.999993  0.999993  0.988897  NaN  0.995163  0.966974  0.998874   
        NcR  0.998867  0.998851  0.994522  NaN  0.998171  0.977345  0.999996   

                  phi      time  
twindow                          
0       BPR -0.779664 -0.668647  
        Fn   0.802226  0.579994  
        MN   0.900469  1.000000  
        Nc   0.960932  0.858232  
        NcR  0.972280  0.868831  

[5 rows x 30 columns]

In [271]:
import itertools
col_list = list(a.columns.values)
col_pairs = [(i,j) for i,j in itertools.product(col_list, col_list) if i != j]

In [284]:
from numpy import tril
a.loc[:,:] =  tril(a, k=-1)
ccor = a.stack()

# Model 1: compare cruise-level parameters for two flights
All records (frequency = second) in the respective two flights are considered. <br>
Records in the first flight are marked with target = 0, others with target = 1. <br>
A classification model is fit. The accuracy of the fit indicates whether a difference can be inferred between the two samples.

In [ ]:
from numpy import array
FLIGHT_1 = 1
FLIGHT_2 = 5

#remove non-numeric fields
mod1 = equip_df.query('fl_num==' + str(FLIGHT_1) + ' or fl_num==' + str(FLIGHT_2))

mod1.ix[FLIGHT_1:FLIGHT_1,'target'] = 0
mod1.ix[FLIGHT_2:FLIGHT_2,'target'] = 1
target = mod1['target']

mod1 = mod1.drop(['time','eq','engine_num','isfault','iscruise','target'], axis=1)

In [ ]:
print 'Training records with target=0: ' + str(mod1[mod1['target']==0].shape[0])
print 'Training records with target=1: ' + str(mod1[mod1['target']==1].shape[0])
print 'Total training records: ' + str(mod1.shape[0])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# fit a CART model to the data
model = DecisionTreeClassifier()
model.fit(mod1, target)

# make predictions
expected = target
predicted = model.predict(mod1)

#put target & predicted values back to 
mod1['target'] = target
mod1['predicted'] = predicted

# summarize the fit of the model
#print(metrics.classification_report(expected, predicted))
#print(metrics.confusion_matrix(expected, predicted))
print 'Accuracy: ' + str(metrics.accuracy_score(expected, predicted) * 100) + '%'

Although the accuracy of fit is high, no explanatory variables seems to correlate well with the target.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from numpy import diag

#print mod1.columns
print mod1.corr()['predicted']
#mod1[['alt','predicted']].plot(subplots=True)

plt.matshow(diag(mod1.corr()['predicted']))

In [ ]:
#wSpec = Window.orderBy("time")
#a.select('time', fsql.lag('time', 1).over(wSpec).alias('time_lagged')).take(100)